# _Nested sampling_ avec _dynesty_

## Prior transforms

Le concept de _prior transform_ est un peu moins intuitif que celui d'une distribution à priori (_prior_) comme celle utilisée dans un MCMC. Dans un MCMC, la distribution $\pi(\theta)$ est définie **explicitement** en fonction du vecteur de paramètres $\theta$.

Dans le cas du _nested sampling_, les échantillons sont toujours tirés d'un espace uniforme. Il faut donc spécifier une transformation du cube unitaire en $D$ dimensions vers l'espace de paramètres qui nous intéresse (par exemple, un paramètre uniforme entre -100 et 100, un autre gaussien centré sur 0 avec un écart-type de 10). Pour des paramètres indépendants, il suffit d'utilier la distribution cumulative inverse, plus souvent appelée "fonction quantile" ou encore "_percent point function_", en anglais. Il est aussi possible de définir des corrélations ou des conditions entre les paramètres. La [section dédiée aux prior transforms](https://dynesty.readthedocs.io/en/latest/quickstart.html#prior-transforms) de la documtentation de _dynesty_ donne plus d'information à ce sujet.

Pour explorer un peu plus les _prior transforms_, nous allons considérer deux paramètres: l'un ayant un prior uniforme $U(-10, 10)$ et un autre ayant un prior gaussien $\mathcal{N}(5.0, 1.0)$. En général, un _prior transform_ uniforme est assez simple à implémenter manuellement: il suffit de le re-centrer et de le multiplier pour avoir la bonne largeur (car le point de départ est déjà uniforme, mais sur l'intervalle $U(0, 1)$). Pour la majorité des autres distributions (par exemple une gaussienne ou une distribution uniforme en log), le plus simple sera d'utiliser `scipy.stats` avec la fonction `ppf()`. L'exemple plus bas utilise cette méthode pour la distribution gaussienne.

Pour ce premier exemple, on ne fera aucun échantillonage. Le but est juste de vérifier que nos fonctions de _prior transform_ font ce à quoi on s'attend (en général, c'est une bonne habitude de faire cette vérification la première fois qu'on utilise une distribution si on a un doute).

Commençons donc par définir notre fonction `prior_transform()`.

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import rcParams

plt.style.use('tableau-colorblind10')
rcParams["font.size"] = 18
rcParams["figure.figsize"] = (9, 6)
rcParams["xtick.direction"] = "in"
rcParams["ytick.right"] = True

In [ ]:
import numpy as np
from scipy import stats
from numpy.random import default_rng

def prior_transform(u):
    
    p = np.array(u)
    
    # TODO: Définir les transformations
    
    return p

Générons d'abord des points sur un carré unitaire et vérifion la distribution avec `corner`.

Maintenant, `prior_transform()` devrait transformer les distributions $U(0, 1)$ vers notre espace de paramètre définit plus haut.

Comme prévu, les distributions sont transformées vers nos _priors_ !

## Exemple de comparaison de modèle.

Supposons qu'on obtient des observations quelconques. On veut savoir si les données sont mieux représentées par une parabole ou bien si une droite suffit. On peut utiliser les mêmes données simulées que dans le tutoriel `emcee` de la semaine dernière.

In [ ]:
np.random.seed(123)

# Choose the "true" parameters.
m_true = -0.9594
b_true = 4.294
f_true = 0.534

# Generate some synthetic data from the model.
N = 50
x = np.sort(10 * np.random.rand(N))
yerr = 0.1 + 0.5 * np.random.rand(N)
y = m_true * x + b_true
y += yerr * np.random.randn(N)

plt.errorbar(x, y, yerr=yerr, fmt=".k", capsize=0)
x0 = np.linspace(0, 10, 500)
plt.plot(x0, m_true * x0 + b_true, "k", alpha=0.3, lw=3)
plt.xlim(0, 10)
plt.xlabel("x")
plt.ylabel("y")
plt.show()

### Model 1: Droite

On teste d'abord la droite en utilisant les mêmes _priors_ que dans le tutoriel d'emcee (en général on voudrait un _prior_ uniforme en log pour les paramètres d'amplitude, mais une distribution uniforme fera l'affaire ici).

In [ ]:
ndim = 2

def log_likelihood(theta, x, y, yerr):
    # TODO: Log likelihood
    pass

In [ ]:
def ptform_line(u):
    
    p = np.array(u)
    
    # TODO: Générer les transformations
    
    return p

Une fois le modèle et le _prior_ définis, on peut utiliser `dynesty` pour faire un _nested sampling_ dynamique.

In [ ]:
import dynesty

# TODO: Utiliser dynesty pour exécuter un Nested sampling dynamique

Les graphiques inclus dans dynesty peuvent nous informer sur l'échantillonage et sur l'évolution des points.

In [ ]:
from dynesty import plotting as dyplot

# TODO: Utiliser dynesty pour afficher différents graphqiques

Ensuite, on peut calculer l'évidence et d'autres informations qui nous intéressent.

In [ ]:
from dynesty import utils as dyfunc

# TODO: Calculer les valeurs des paramètres (quantiles) et de l'évidence

In [ ]:
quantiles = np.array(quantiles)
for i in range(ndim):
    print(quantiles[i, 1], "+", quantiles[i, 2] - quantiles[i, 1], "-", quantiles[i, 1] - quantiles[i, 0])

In [ ]:
m_line, b_line = quantiles[:, 1]

In [ ]:
plt.errorbar(x, y, yerr=yerr, fmt=".k", capsize=0)
x0 = np.linspace(0, 10, 500)
plt.plot(x0, m_true * x0 + b_true, "k", alpha=0.3, lw=3)
plt.plot(x0,  m_line* x0 + b_line, "b", alpha=0.3, lw=3)
plt.xlim(0, 10)
plt.xlabel("x")
plt.ylabel("y")
plt.show()

### Model 2: Parabole
On peut maintenant répéter toute la procédure avec une parabole au lieu d'une droite. Par inspection visuelle, il semble que garder le premier coefficient entre -0.1 et 0.1 soit bien suffisant. On utilisera donc ce _prior_ uniforme.

In [ ]:
ndim = 3

def log_likelihood(theta, x, y, yerr):
    # TODO: parabole
    pass

In [ ]:
def ptform(u):
    
    p = np.array(u)
    
    # TODO: Transformer les parmètres
    
    return p

In [ ]:
import dynesty

# TODO: Effectuer le nested sampling pour la parabole

In [ ]:
# TODO: Générer les graphiques

In [ ]:
from dynesty import utils as dyfunc

# TODO: Calculer les paramètres et l'évidence

In [ ]:
quantiles = np.array(quantiles)
for i in range(ndim):
    print(quantiles[i, 1], "+", quantiles[i, 2] - quantiles[i, 1], "-", quantiles[i, 1] - quantiles[i, 0])

In [ ]:
a_para, m_para, b_para = quantiles[:, 1]

In [ ]:
plt.errorbar(x, y, yerr=yerr, fmt=".k", capsize=0)
x0 = np.linspace(0, 10, 500)
plt.plot(x0, m_true * x0 + b_true, "k", alpha=0.3, lw=3)
plt.plot(x0,  m_line* x0 + b_line, "b", alpha=0.3, lw=3)
plt.plot(x0,  a_para * x0**2 + m_para* x0 + b_para, "r", alpha=0.3, lw=3)
plt.xlim(0, 10)
plt.xlabel("x")
plt.ylabel("y")
plt.show()

### Comparaison des modèles

On peut maintenat utiliser le tableau de Trotta (2008) pour comparer les modèles:

<img src="table1_trotta.png" width=500 height=400 />

Grâce aux évidences calculées ci-haut, on a:

In [ ]:
# TODO: Calculer le Bayes factor